In [1]:
import pandas as pd
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier

dataframe = pd.read_csv("Frases_Emojis.csv")
dataframe

,Frase,emoji
0,Tengo hambre,🍔
1,Estoy muerto de hambre,🍔
2,No aguanto más necesito comer,🍔
3,Mi estómago está rugiendo,🍔
4,Podría comerme una vaca,🍔
...,...,...
168,No aguanto ese olor,🤮
169,Que olor tan desagradable,🤮
170,Esto está podrido,🤮
171,Me dan escalofríos,🤮


Seleccionamos las frases para transformarlas a vector con TF-IDF

In [2]:
#dataframe["Frase"]
tfidf_vectorizer = TfidfVectorizer()
x_train = tfidf_vectorizer.fit_transform(dataframe["Frase"]) 

Entrena el modelo knn 

In [3]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(x_train, dataframe["emoji"])

KNeighborsClassifier(n_neighbors=3)

Primero hice una prueba unitaria para ver que funcionara con una frase muy "obvia"

In [4]:
nuevo_dato = ["Eso suena delicioso"]
nuevo_dato_vect = tfidf_vectorizer.transform(nuevo_dato)

predict = knn.predict(nuevo_dato_vect)
print(predict[0])

😋


Ahora hice una prueba con diferentes frases y los resultados fueron muy curiosos

In [5]:
prueba = ["Qué hambre", "Me duele el estómago", "Podría comerme un banquete entero", "Hoy todo me fue increíble", "Que día tan increíble", "Hoy todo fue genial", "Eso suena como un manjar", "Estoy salivando de imaginarlo", "Estoy enamorado", "Te quiero demasiado", "Te mando un gran beso", "Jajaja no puedo parar de reír", "Jajajajajaja", "Como si me lo fuera a creer", "Eso suena súper falso", "Eso es más aburrido que nada", "Que sencillo", "En serio pasó de verdad?", "No puede ser", "Suena increíble", "Siempre lo que hago sale bien", "No dudes de mí, mira y aprende", "Me tienes muy molesto", "Me sacaste de quicio", "Quedé como payaso", "Tierra, trágame y no me devuelvas jamás", "Las manos me sudan", "Esto me pone tenso", "Ni idea", "No me quita el sueño", "No lo sé ni me importa", "No podría importarme menos", "Esto me da demasiado asco", "Eso fue súper desagradable"]
prueba_vect = tfidf_vectorizer.transform(prueba)

for i in range (len(prueba)):
    prueba_predict = knn.predict(prueba_vect[i])
    print(f"{i}: {prueba[i]} -> {prueba_predict}")

0: Qué hambre -> ['🍔']
1: Me duele el estómago -> ['🍔']
2: Podría comerme un banquete entero -> ['🍔']
3: Hoy todo me fue increíble -> ['😄']
4: Que día tan increíble -> ['😄']
5: Hoy todo fue genial -> ['😎']
6: Eso suena como un manjar -> ['😋']
7: Estoy salivando de imaginarlo -> ['🍔']
8: Estoy enamorado -> ['😄']
9: Te quiero demasiado -> ['🤡']
10: Te mando un gran beso -> ['😋']
11: Jajaja no puedo parar de reír -> ['🤣']
12: Jajajajajaja -> ['🍔']
13: Como si me lo fuera a creer -> ['😮']
14: Eso suena súper falso -> ['😋']
15: Eso es más aburrido que nada -> ['🙄']
16: Que sencillo -> ['😬']
17: En serio pasó de verdad? -> ['😮']
18: No puede ser -> ['😮']
19: Suena increíble -> ['😋']
20: Siempre lo que hago sale bien -> ['😑']
21: No dudes de mí, mira y aprende -> ['🍔']
22: Me tienes muy molesto -> ['😬']
23: Me sacaste de quicio -> ['🍔']
24: Quedé como payaso -> ['🍔']
25: Tierra, trágame y no me devuelvas jamás -> ['🤡']
26: Las manos me sudan -> ['😄']
27: Esto me pone tenso -> ['🤮']
28: Ni ide

Vemos que la 🍔 que debería simbolizar el hambre fue interpretado en muchas frases que no tiene nada que ver, podría deberse a la variedad de palabras que tiene el dataset. 

Vemos que algunas frases tienen un emoji relacionado a algo que no encuentro el sentido, los mostraré a continuación y en algunos casos escribiré al final la palabra que creo hizo que creyera que se trataba de ese emoji, lo cual se debió a algún sesgo por las "pocas" frases que tiene el dataset de entrenamiento:

* Estoy enamorado -> ['😄'] 
* Te quiero demasiado -> ['🤡'] (Esta me dió mucha risa)
* Te mando un gran beso -> ['😋'] - Gran
* Jajajajajaja -> ['🍔']
* Eso suena súper falso -> ['😋'] - suena
* Que sencillo -> ['😬'] 
* Suena increíble -> ['😋'] - suena
* No dudes de mí, mira y aprende -> ['🍔']
* Me tienes muy molesto -> ['😬']
* Me sacaste de quicio -> ['🍔']
* Las manos me sudan -> ['😄'] (también podría quedar bien según el contexto)
* Eso fue súper desagradable -> ['😋'] (Eso?)

Hubieron otras que dieron un resultado diferente al esperado, pero que aún así tenían cierto sentido:
* Hoy todo fue genial -> ['😎'] - Original: 😄. Probablemente fue por contener la palabra genial
* Siempre lo que hago sale bien -> ['😑'] - Original:😎. Lo curioso es que queda bien si fuera en un sentido irónico
* Esto me pone tenso -> ['🤮'] - Original:😬. Igualmente muestra una emoción de desagrado
* No me quita el sueño -> ['😎'] - Original:🤷‍♂️. Era para indiferencia pero muestra seguridad a pesar de cierta situación
* Esto me da demasiado asco -> ['😑']. Original:🤮

Veo que no logró reconocer de ninguno de los siguientes emojis: .
Por lo que intentaré forzarlos con otra prueba que busque su aparición: 🥰, 😡. 

In [9]:
prueba = ["Eres un amor", "Me gustas mucho", "Te amo mucho", "Eres mi gran amor", "Que coraje", "Me estás hartando", "Me tienes harto", "Me estás sacando de quicio"]
prueba_vect = tfidf_vectorizer.transform(prueba)

for i in range (len(prueba)):
    prueba_predict = knn.predict(prueba_vect[i])
    print(f"{i}: {prueba[i]} -> {prueba_predict}")

0: Eres un amor -> ['🥰']
1: Me gustas mucho -> ['🙄']
2: Te amo mucho -> ['🥰']
3: Eres mi gran amor -> ['🥰']
4: Que coraje -> ['😡']
5: Me estás hartando -> ['😡']
6: Me tienes harto -> ['🙄']
7: Me estás sacando de quicio -> ['😡']
